# Deploy a TSFM Inference Endpoint on Amazon Sagemaker


In this notebook, we walk through the process of deploying an IBM granite timeseries inference service enpoint on Amazon SageMaker.

In [87]:
# obtain tsfm requirements
import shutil
import subprocess
from pathlib import Path


# so sagemaker installs all the tsfm pre-reqs
subprocess.check_output(
    [
        "curl",
        "-o",
        "code/requirements.txt",
        "https://raw.githubusercontent.com/ibm-granite/granite-tsfm/refs/heads/sagemaker/services/inference/requirements.txt",
    ]
)

# so that some code directories from the granite-tsfm repo can be added to the sys.path
shutil.rmtree("code/granite-tsfm", ignore_errors=True)
assert not Path("code/granite-tsfm").exists()
subprocess.check_call(
    cwd="code",
    args=[
        "git",
        "clone",
        "-b",
        "sagemaker",
        "--depth",
        "1",
        "https://github.com/ibm-granite/granite-tsfm.git",
    ],
)
# set up tsfm services layer
# 1. boilerplate code
subprocess.check_call(cwd="code/granite-tsfm/services/inference", args=["make", "boilerplate"])
# 2. tsfm-granite public models
# you must have git-lfs installed for this
subprocess.check_call(args=["git-lfs"])
subprocess.check_call(cwd="code/granite-tsfm/services/inference", args=["make", "clone_models"])

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5944  100  5944    0     0  21855      0 --:--:-- --:--:-- --:--:-- 21933
Cloning into 'granite-tsfm'...


rm tsfminference/.gitignore || true
echo "# THIS FILE IS AUTOMATICALLY GENERATED, YOUR CHANGES WILL BE OVERWRITTEN" > tsfminference/.gitignore
for f in ../boilerplate/*.py; do \
	echo $f; \
	cat ../boilerplate/warning.txt > tsfminference/$(basename $f); \
	cat $f>>tsfminference/$(basename $f); \
	echo $(basename $f) >> tsfminference/.gitignore; \
	done 
../boilerplate/dataframe_checks.py
../boilerplate/dirutil.py
../boilerplate/errors.py
../boilerplate/hfutil.py
../boilerplate/inference_payloads.py
git-lfs/3.6.0 (GitHub; linux amd64; go 1.23.1; git 6340befc)
git lfs <command> [<args>]

Git LFS is a system for managing and versioning large files in
association with a Git repository. Instead of storing the large files
within the Git repository as blobs, Git LFS stores special "pointer
files" in the repository, while storing the actual file contents on a
Git LFS server. The contents of the large file are downloaded
automatically when needed, for example when a Git branch containing the
la

Cloning into 'mytest-tsfm'...
Filtering content: 100% (15/15), 104.18 MiB | 26.09 MiB/s, done.


0

In [93]:
# obtain the mdhash of our repo so that models and endpoints can use that in their name
result = subprocess.run(
    cwd="code/granite-tsfm/services/inference",
    args=["git", "rev-parse", "--short", "HEAD"],
    capture_output=True,
    text=True,
)
tsfm_release = f"tsfm-public-{result.stdout.strip()}"

result = subprocess.run(
    cwd="code/granite-tsfm/services/inference/mytest-tsfm",
    args=["git", "rev-parse", "--short", "HEAD"],
    capture_output=True,
    text=True,
)
tsfm_model_release = f"model-release-{result.stdout.strip()}"
identifier = f"{tsfm_release}-{tsfm_model_release}"
identifier


'tsfm-public-95ed712-model-release-fc98672'

In [94]:
import subprocess

import boto3
from sagemaker import Session
from sagemaker.pytorch import PyTorchModel


# Make sure you have run the aws cli command "aws config" to set up your system correctly
# with your aws credentials

boto3_session = boto3.Session()

sess = Session(boto_session=boto3_session)
role = "arn:aws:iam::481118440516:role/SagemakerFullAccessRole"

# create the tarball of code and model artifacts
subprocess.check_call(
    [
        "tar",
        "--exclude",
        "*.git/**",
        "-czvf",
        "model.tar.gz",
        "code/inference.py",
        "code/requirements.txt",
        "code/granite-tsfm/services/inference",
    ]
)

# upload it to our default buucket sot that sagemaker get get it
tsfm_tarball = sess.upload_data(path="model.tar.gz", bucket=sess.default_bucket(), key_prefix="model/pytorch")

[12/23/24 10:14:11] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=545004;file:///home/stus/py311/lib64/python3.11/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=1251;file:///home/stus/py311/lib64/python3.11/site-packages/botocore/credentials.py#1278\1278]8;;\

code/inference.py
code/requirements.txt
code/granite-tsfm/services/inference/
code/granite-tsfm/services/inference/mytest-tsfm/
code/granite-tsfm/services/inference/mytest-tsfm/ttm-1536-96-r2/
code/granite-tsfm/services/inference/mytest-tsfm/ttm-1536-96-r2/generation_config.json
code/granite-tsfm/services/inference/mytest-tsfm/ttm-1536-96-r2/training_args.bin
code/granite-tsfm/services/inference/mytest-tsfm/ttm-1536-96-r2/config.json
code/granite-tsfm/services/inference/mytest-tsfm/ttm-1536-96-r2/tsfm_config.json
code/granite-tsfm/services/inference/mytest-tsfm/ttm-1536-96-r2/model.safetensors
code/granite-tsfm/services/inference/mytest-tsfm/ttm-1536-96-r2/README.md
code/granite-tsfm/services/inference/mytest-tsfm/chronos-t5-tiny/
code/granite-tsfm/services/inference/mytest-tsfm/chronos-t5-tiny/generation_config.json
code/granite-tsfm/services/inference/mytest-tsfm/chronos-t5-tiny/figures/
code/granite-tsfm/services/inference/mytest-tsfm/chronos-t5-tiny/figures/main-figure.png
code/gra

## PyTorch Model Object

The `PyTorchModel` class allows you to define an environment for making inference using your
model artifact. Like the `PyTorch` class discussed 
[in this notebook for training an PyTorch model](get_started_mnist_train.ipynb), it is a high level API used to set up a docker image for your model hosting service.

Once it is properly configured, it can be used to create a SageMaker
endpoint on an EC2 instance. The SageMaker endpoint is a containerized environment that uses your trained model 
to make inference on incoming data via RESTful API calls. 

Some common parameters used to initiate the `PyTorchModel` class are:
- `entry_point`: A user defined python file to be used by the inference image as handlers of incoming requests
- `source_dir`: The directory of the `entry_point`
- `role`: An IAM role to make AWS service requests
- `model_data`: the S3 location of the compressed model artifact. It can be a path to a local file if the endpoint 
is to be deployed on the SageMaker instance you are using to run this notebook (local mode)
- `framework_version`: version of the PyTorch package to be used
- `py_version`: python version to be used

We elaborate on the `entry_point` below.



In [95]:
model = PyTorchModel(
    entry_point="inference.py",
    source_dir="code",
    role=role,
    model_data=tsfm_tarball,
    framework_version="2.5",
    py_version="py311",
    name=f"model-{identifier}",
)

## Deploy the inference container
Once the `PyTorchModel` class is initiated, we can call its `deploy` method to run the container for the hosting
service. Some common parameters needed to call `deploy` methods are:

- `initial_instance_count`: the number of SageMaker instances to be used to run the hosting service.
- `instance_type`: the type of SageMaker instance to run the hosting service. Set it to `local` if you want to run the hosting service on the local SageMaker instance. Local mode is typically used for debugging. 
- `serializer`: A python callable used to serialize (encode) the request data.
- `deserializer`: A python callable used to deserialize (decode) the response data.

Commonly used serializers and deserializers are implemented in `sagemaker.serializers` and `sagemaker.deserializers`
submodules of the SageMaker Python SDK. 

Since in the `transform_fn` we declared that the incoming requests are json-encoded, we need to use a `json serializer`,
to encode the incoming data into a json string.

In [96]:
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer


instance_type = "ml.c4.xlarge"

# this could take up to five minutes to complete
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
    endpoint_name=f"endpoint-{identifier}",
)
predictor

[12/23/24 10:14:28] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=865146;file:///home/stus/py311/lib64/python3.11/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=409127;file:///home/stus/py311/lib64/python3.11/site-packages/botocore/credentials.py#1278\1278]8;;\

[12/23/24 10:14:29] INFO     Repacking model artifact                                                  ]8;id=771438;file:///home/stus/py311/lib64/python3.11/site-packages/sagemaker/model.py\model.py]8;;\:]8;id=854367;file:///home/stus/py311/lib64/python3.11/site-packages/sagemaker/model.py#819\819]8;;\
                             (s3://sagemaker-us-east-2-481118440516/model/pytorch/model.tar.gz),                   
                             script artifact (code), and dependencies ([]) into single tar.gz file                 
                             located at                                                                            
                             s3://sagemaker-us-east-2-481118440516/model-tsfm-public-95ed712-model-rel             
                             ease-fc98672/model.tar.gz. This may take some time depending on model                 
                             size...                                                                               

[12/23/24 10:15:09] INFO     Creating model with name:                                              ]8;id=339662;file:///home/stus/py311/lib64/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=96724;file:///home/stus/py311/lib64/python3.11/site-packages/sagemaker/session.py#4094\4094]8;;\
                             model-tsfm-public-95ed712-model-release-fc98672                                       

[12/23/24 10:15:10] INFO     Creating endpoint-config with name                                     ]8;id=929071;file:///home/stus/py311/lib64/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=147899;file:///home/stus/py311/lib64/python3.11/site-packages/sagemaker/session.py#5889\5889]8;;\
                             endpoint-tsfm-public-95ed712-model-release-fc98672                                    

                    INFO     Creating endpoint with name                                            ]8;id=282436;file:///home/stus/py311/lib64/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=632089;file:///home/stus/py311/lib64/python3.11/site-packages/sagemaker/session.py#4711\4711]8;;\
                             endpoint-tsfm-public-95ed712-model-release-fc98672                                    

-------!

The `predictor` we get above can be used to make prediction requests against a SageMaker endpoint. 
For more information, check [the API reference for SageMaker Predictor](
https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html#sagemaker.predictor.predictor)

Now, let's test the endpoint with some dummy data.

We'll create a univariate single (all given with the ID "A") timeseries.

In [76]:
from datetime import datetime

import numpy as np
import pandas as pd


tslength = 1024
start_date = datetime(2020, 1, 1)
date_range = [d.isoformat() for d in pd.date_range(start=start_date, periods=tslength, freq="h")]
ids = ["A" for _ in range(tslength)]
values = np.random.rand(tslength)

# Create the DataFrame
df = pd.DataFrame({"date": date_range, "ID": ids, "value": values})

df[0:10]

,date,ID,value
0,2020-01-01T00:00:00,A,0.538112
1,2020-01-01T01:00:00,A,0.484820
2,2020-01-01T02:00:00,A,0.404628
3,2020-01-01T03:00:00,A,0.497913
4,2020-01-01T04:00:00,A,0.877828
5,2020-01-01T05:00:00,A,0.486114
6,2020-01-01T06:00:00,A,0.724446
7,2020-01-01T07:00:00,A,0.291108
8,2020-01-01T08:00:00,A,0.154599
9,2020-01-01T09:00:00,A,0.937216


Prepare the payload

In [ ]:
payload = {
    "inference_type": "forecasting",  # we currently support only 'forecasting'
    "model_id": "ttm-1024-96-r1",
    "parameters": {},
    "schema": {
        "timestamp_column": "date",
        "id_columns": ["ID"],  # multiple columns are supported
        "target_columns": ["value"],  # what we're generating a forecast for
    },
    "data": df.to_dict(orient="list"),
    "future_data": {},  # used for things like exogenous data
}
payload

{'inference_type': 'forecasting',
 'model_id': 'ttm-1024-96-r1',
 'parameters': {},
 'schema': {'timestamp_column': 'date',
  'id_columns': ['ID'],
  'target_columns': ['value']},
 'data': {'date': ['2020-01-01T00:00:00',
   '2020-01-01T01:00:00',
   '2020-01-01T02:00:00',
   '2020-01-01T03:00:00',
   '2020-01-01T04:00:00',
   '2020-01-01T05:00:00',
   '2020-01-01T06:00:00',
   '2020-01-01T07:00:00',
   '2020-01-01T08:00:00',
   '2020-01-01T09:00:00',
   '2020-01-01T10:00:00',
   '2020-01-01T11:00:00',
   '2020-01-01T12:00:00',
   '2020-01-01T13:00:00',
   '2020-01-01T14:00:00',
   '2020-01-01T15:00:00',
   '2020-01-01T16:00:00',
   '2020-01-01T17:00:00',
   '2020-01-01T18:00:00',
   '2020-01-01T19:00:00',
   '2020-01-01T20:00:00',
   '2020-01-01T21:00:00',
   '2020-01-01T22:00:00',
   '2020-01-01T23:00:00',
   '2020-01-02T00:00:00',
   '2020-01-02T01:00:00',
   '2020-01-02T02:00:00',
   '2020-01-02T03:00:00',
   '2020-01-02T04:00:00',
   '2020-01-02T05:00:00',
   '2020-01-02T06:00:00'

Use the predictor to generate a forecast

In [ ]:
# import json
res = predictor.predict(payload)
res

{'model_id': 'ttm-1024-96-r1',
 'created_at': '2024-12-23T14:38:28.778599',
 'results': [{'date': ['2020-02-12T16:00:00',
    '2020-02-12T17:00:00',
    '2020-02-12T18:00:00',
    '2020-02-12T19:00:00',
    '2020-02-12T20:00:00',
    '2020-02-12T21:00:00',
    '2020-02-12T22:00:00',
    '2020-02-12T23:00:00',
    '2020-02-13T00:00:00',
    '2020-02-13T01:00:00',
    '2020-02-13T02:00:00',
    '2020-02-13T03:00:00',
    '2020-02-13T04:00:00',
    '2020-02-13T05:00:00',
    '2020-02-13T06:00:00',
    '2020-02-13T07:00:00',
    '2020-02-13T08:00:00',
    '2020-02-13T09:00:00',
    '2020-02-13T10:00:00',
    '2020-02-13T11:00:00',
    '2020-02-13T12:00:00',
    '2020-02-13T13:00:00',
    '2020-02-13T14:00:00',
    '2020-02-13T15:00:00',
    '2020-02-13T16:00:00',
    '2020-02-13T17:00:00',
    '2020-02-13T18:00:00',
    '2020-02-13T19:00:00',
    '2020-02-13T20:00:00',
    '2020-02-13T21:00:00',
    '2020-02-13T22:00:00',
    '2020-02-13T23:00:00',
    '2020-02-14T00:00:00',
    '2020-02-1

In [74]:
# note that json return-type inference.py is deserialized automatically to dictionary
type(res)

dict

### Todo

1. make a call via requests or curl
2. document installation procedure

## (Optional) Clean up 

If you do not plan to use the endpoint, you should delete it to free up some computation 
resource. If you use local, you will need to manually delete the docker container bounded
at port 8080 (the port that listens to the incoming request).


In [97]:
predictor.delete_model()
predictor.delete_endpoint()

[12/23/24 10:19:54] INFO     Deleting model with name:                                              ]8;id=811845;file:///home/stus/py311/lib64/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=385148;file:///home/stus/py311/lib64/python3.11/site-packages/sagemaker/session.py#5226\5226]8;;\
                             model-tsfm-public-95ed712-model-release-fc98672                                       

                    INFO     Deleting endpoint configuration with name:                             ]8;id=692755;file:///home/stus/py311/lib64/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=70350;file:///home/stus/py311/lib64/python3.11/site-packages/sagemaker/session.py#4865\4865]8;;\
                             endpoint-tsfm-public-95ed712-model-release-fc98672                                    

[12/23/24 10:19:55] INFO     Deleting endpoint with name:                                           ]8;id=682710;file:///home/stus/py311/lib64/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=830431;file:///home/stus/py311/lib64/python3.11/site-packages/sagemaker/session.py#4855\4855]8;;\
                             endpoint-tsfm-public-95ed712-model-release-fc98672                                    